# Práctica 1

## Preparación del entorno

In [7]:
import numpy as np
import tensorflow as tf
import keras as ker
import matplotlib.pyplot as plt 
import pandas as pd
import sklearn as sk
import PIL
import os
import random

from PIL import Image
from IPython.display import clear_output # Limpiar output
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder
from numpy import asarray
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm

# Preprocesamiento de datos

In [18]:
def crear_data_set_entreno(): # Besitos gonzalo de parte de Ini y Marcos <3
# Creamos un set entero de entrenamiento. A posterior este set lo dividiremos en entrenamiento y validación

# Esto es para decir como queremos las nuevas imagenes
    augmentation = ImageDataGenerator( # Esto lo vamos a tener que poner todo seguido sin comentar para que no salga plagio !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        rescale=1/255.,              # normalizar                     > Es posible que esto no sea necesario ya que ya esta normalizado
        brightness_range=[0.1, 0.7], # luminosidad
        width_shift_range=0.5,       # cambiar anhcura 50%
        rotation_range=0,            # rotar
        shear_range = 0.2,           # idk
        zoom_range=0.2,              # zoom
        horizontal_flip=True,        # 180º horizontal
        vertical_flip=True,          # 180º vertical
        validation_split=0.2        # usamos el 20% de los datos por epoch
        )
    # Esto creaa dichas imagenes apartir del directorio que queramos

    augmentation.flow_from_directory(
            directory=r"../Dataset/imgs/train/",
            target_size=(128, 96), #Queremos tener imagenes de la misam resolucion
            color_mode="rgb",
            batch_size=15, # N_img / batch_size = N_nuevas_imagenes
            class_mode="categorical",
            shuffle=True,
            seed=42
    )

    # Dividimos las imagenees nuevas en 2 sets
    dataset_entreno_augmentation = augmentation ### Nos falta añadir las de entreno aparte de las obtenidas por augmentation
    entreno_augmentation, validacion_augmentation = train_test_split(dataset_entreno_augmentation, test_size = 0.20, shuffle=True) # El entreno es el 80% y el conjunto prueba 20% para el set de augmentation


    # Ahora creamos el set de entrenamiento con las imagenes originales
    entrenamiento = tf.keras.utils.image_dataset_from_directory(
    directory = r"../Dataset/imgs/train/",
    seed=123,
    image_size=(128, 96),
    batch_size=32
    )
    # Dividimos dicho set en dos
    dataset_entreno_original = entrenamiento ### Nos falta añadir las de entreno aparte de las obtenidas por augmentation
    entreno_original, validacion_original = train_test_split(dataset_entreno_original, test_size = 0.20, shuffle=True) # El entreno es el 80% y el conjunto prueba 20% para el set de augmentation

    # Juntamos los sets de augmentation (las imagenes nuevas) con el set de las originales (las que nos dan)
    entreno = entreno_original.append(entreno_augmentation)
    validacion = validacion_original.append(validacion_augmentation)

    return entreno,validacion

entreno , validacion = crear_data_set_entreno()

Found 22424 images belonging to 10 classes.


TypeError: Singleton array array(<keras.preprocessing.image.ImageDataGenerator object at 0x000001C04478B790>,
      dtype=object) cannot be considered a valid collection.

### Variables a definir

In [ ]:
# No tocar
num_clases = 10
# La foto es 128x96
xpixel = 128
ypixel = 96
# Tocar
n_neuronas_conv1 = 32
n_neuronas_conv2 = 64
n_neuronas_conv3 = 128
l_rate = 0.1
epoch = 12 
batch = 128

##### Otra Red para no borrar lo ya hecho

In [ ]:
def Modelar_red(n_conv1,n_conv2,n_conv3,x,y,lr):
    model = Sequential()

    #### Input Layer ####
    model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same',
                    activation='relu', input_shape=(128, 96, 3)))

    #### Convolutional Layers ####
    model.add(Conv2D(32, (3,3), activation='relu'))
    model.add(MaxPooling2D((2,2)))  # Pooling
    model.add(Dropout(0.2)) # Dropout

    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), activation='relu'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(512, (5,5), padding='same', activation='relu'))
    model.add(Conv2D(512, (5,5), activation='relu'))
    model.add(MaxPooling2D((4,4)))
    model.add(Dropout(0.2))

    #### Fully-Connected Layer ####
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(len(class_subset), activation='softmax'))

    # Compilamos
    adam = ker.optimizers.Adam(learning_rate= lr)
    model.compile(loss=ker.loss.categorical_crossentropy,optimizer=adam,metrics=['accuracy'])

    model.summary() # a handy way to inspect the architecture
    return model

In [ ]:
def Modelar_red(n_conv1,n_conv2,n_conv3,x,y,lr):
    model=Sequential()
    # Capa input
    model.add(Conv2D(filters= n_conv1, kernel_size=(3,3),input_shape=(x,y,3),padding='same',activation='relu',kernel_constraint=maxnorm(3))) 
    # Capas convolucionales
    #   >este bloque se puede seguir añadiendo, quiza con menos neuronas, o menos capas convolucionales, pongo dos por dar un ejemplo nada mas
    # (3,3) es mucho se recomienda unsar 1x1 cuando las img no son mayores de 128x128 la nuestra es 128x96
    model.add(Conv2D(n_conv2,(3,3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2))) # pooling
    model.add(Dropout(0.2)) # dropout

    model.add(Conv2D(n_conv3,(3,3),activation='relu',padding='same')) 
    model.add(Conv2D(n_conv3,(3,3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2))) 
    model.add(Dropout(0.2)) 

    # Capa fully-connected - MLP
    model.add(Flatten())
    model.add(Dense(512,activation='relu',kernel_constraint=maxnorm(3))) # red fully-connected
    model.add(Dropout(0.5))
    model.add(Dense(num_clases, activation='softmax')) # capa de salida(softmaxx) 

    print(model.summary()) # con esto inspeccionamos el modelo, muy comodo

    # Compilamos
    adam = ker.optimizers.Adam(learning_rate= lr)
    model.compile(loss=ker.loss.categorical_crossentropy,optimizer=adam,metrics=['accuracy'])

    return model
model = Modelar_red(n_neuronas_conv1,n_neuronas_conv2,n_neuronas_conv3,xpixel,ypixel,l_rate)

2022-03-23 17:02:16.654563: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-23 17:02:16.656906: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-23 17:02:16.715503: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


KeyboardInterrupt: 

### Y ahora realizamos el fit para entrenar la red

In [ ]:
def Entrenar(m,e,v,epo,b): #(model,entrenamiento,validacion, epoch,batch)
        m.fit(generator = e,validation_data=v,
                use_multiprocessing=True,workers=6, # Esta parte es para que se separen 6 threads paralelos gracias a fit_generator
                epochs= epo,batch_size = b,verbose=False) 
        return m
model = Entrenar(model,entrenamiento,validacion,epoch,batch)

### Evaluamos el modelo con los datos de test (si hay...)

In [ ]:
def Evaluar(m,e_test,s_test): # model,entrada_test,salida_test
    return m.evaluate(e_test,s_test,verbose=False)
resultado = Evaluar(model, entrada_test, salida_test) # no existen estas variables todavia

### Predecir:

In [ ]:
prediccion = model.predict(x_test) # no existe esta variable todavia
print(prediccion)

# Bibliografía
- https://www.learndatasci.com/tutorials/convolutional-neural-networks-image-classification/
- https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
- https://www.tutorialspoint.com/keras/keras_convolution_neural_network.htm
- https://data-flair.training/blogs/keras-convolution-neural-network/
- https://vijayabhaskar96.medium.com/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720